In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline

pd.set_option('display.max_columns',1000)

In [163]:
train = pd.read_csv('./datasets/train.csv')

# Make Date into datetime object, and index:
train['Date'] = pd.to_datetime(train.Date, format='%Y-%m-%d')
train.set_index('Date', inplace=True, drop=True)
train['Latitude2'] = np.round(train.Latitude,2)
train['Longitude2'] = np.round(train.Longitude,2)
train.head(1)

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Latitude2,Longitude2
Date,,,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,41.95,-87.8


In [3]:
spray = pd.read_csv('./datasets/spray.csv')
# Make Date into datetime object, and index
spray['Date'] = pd.to_datetime(spray.Date, format='%Y-%m-%d')
spray.set_index('Date', inplace=True, drop=True)
spray.drop(columns='Time', inplace=True)
spray.head(1)

,Latitude,Longitude
Date,,
2011-08-29,42.391623,-88.089163


In [8]:
spray_train = spray.merge(train, how='inner', on='Date', suffixes=('_spray','_train'))
spray_train.head(1)

,Latitude_spray,Longitude_spray,spray,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude_train,Longitude_train,AddressAccuracy,NumMosquitos,WnvPresent
Date,,,,,,,,,,,,,,
2013-07-25,41.96052,-87.739783,1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,3,0


In [164]:
spray = pd.read_csv('./datasets/spray.csv')
spray['Time'] = spray['Time'].fillna(value='7:44:32 PM')
spray['DateTime'] = spray.apply(lambda x: str(x['Date'])+' '+str(x['Time']), axis=1)
spray['DateTime'] = spray.apply(lambda x: datetime.strptime(str(x['DateTime']), '%Y-%m-%d %I:%M:%S %p'), axis=1)
spray['Date'] = spray.apply(lambda x: datetime.strptime(str(x['Date']), '%Y-%m-%d'), axis=1)
spray['Latitude2'] = np.round(spray.Latitude,2)
spray['Longitude2'] = np.round(spray.Longitude,2)
spray.set_index('DateTime', inplace=True, drop=True)
spray.head()

,Date,Time,Latitude,Longitude,Latitude2,Longitude2
DateTime,,,,,,
2011-08-29 18:56:58,2011-08-29,6:56:58 PM,42.391623,-88.089163,42.39,-88.09
2011-08-29 18:57:08,2011-08-29,6:57:08 PM,42.391348,-88.089163,42.39,-88.09
2011-08-29 18:57:18,2011-08-29,6:57:18 PM,42.391022,-88.089157,42.39,-88.09
2011-08-29 18:57:28,2011-08-29,6:57:28 PM,42.390637,-88.089158,42.39,-88.09
2011-08-29 18:57:38,2011-08-29,6:57:38 PM,42.390410,-88.088858,42.39,-88.09


In [160]:
spray_train = spray.merge(train, how='left', on='Date', suffixes=('_spray','_train'))

In [214]:
train_spray = train.merge(spray, how='left', on=['Date','Latitude2','Longitude2'], suffixes=('_train','_spray'))
train_spray.set_index('Date',drop=True, inplace=True)
train_spray['IsSpray'] = [0 if x==0 else 1 for x in train_spray.Time.fillna(value=0)]
train_spray.head(1)

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude_train,Longitude_train,AddressAccuracy,NumMosquitos,WnvPresent,Latitude2,Longitude2,Time,Latitude_spray,Longitude_spray,IsSpray
Date,,,,,,,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,41.95,-87.8,NaN,NaN,NaN,0


In [216]:
train_spray.IsSpray.value_counts()

0    10476
1     1944
Name: IsSpray, dtype: int64

In [217]:
train_spray.to_csv(r'./datasets/train_spray_map.csv')